In [ ]:
#Dataset OCT
!gdown --fuzzy "https://drive.google.com/file/d/1cIGCfx6CiVgEpq8PyKzmF1LBJiQGkxzc/view?usp=sharing"

Downloading...
From (original): https://drive.google.com/uc?id=1cIGCfx6CiVgEpq8PyKzmF1LBJiQGkxzc
From (redirected): https://drive.google.com/uc?id=1cIGCfx6CiVgEpq8PyKzmF1LBJiQGkxzc&confirm=t&uuid=adf74847-5251-4393-b75f-feaeb75af915
To: /content/OCT2017.tar.gz
100% 5.79G/5.79G [03:28<00:00, 27.8MB/s]


In [ ]:
#Extract the dataset
!tar -xzf "/content/OCT2017.tar.gz" -C /content/data/

In [ ]:
#Teacher Restnet101
!gdown --fuzzy "https://drive.google.com/file/d/10P6tusq84bi7dPfSSFMbQH03KWOWR9XQ/view?usp=sharing"

Downloading...
From (original): https://drive.google.com/uc?id=10P6tusq84bi7dPfSSFMbQH03KWOWR9XQ
From (redirected): https://drive.google.com/uc?id=10P6tusq84bi7dPfSSFMbQH03KWOWR9XQ&confirm=t&uuid=ab04d8af-5c3f-4261-b65d-45ffddc4bfe6
To: /content/best_resnet101_teacher.pth
100% 525M/525M [00:18<00:00, 28.1MB/s]


In [ ]:
# =================================================================================
# PHASE 2: FEATURE-BASED KNOWLEDGE DISTILLATION - STUDENT MODEL TRAINING (MobileNetV3)
# Using Cosine Similarity Loss
# ==================================================================================================================================================================

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torchvision.models as models
from pathlib import Path
import numpy as np
from tqdm.auto import tqdm
from sklearn.metrics import precision_recall_fscore_support, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
import gc
import warnings
warnings.filterwarnings('ignore')

# Import dataset class from Phase 1
from PIL import Image
from torch.utils.data import Dataset

class MultiClassOCTDataset(Dataset):
    def __init__(self, image_paths, labels, class_names, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.class_names = class_names
        self.transform = transform
        self.class_to_idx = {cls: i for i, cls in enumerate(class_names)}

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label = self.labels[idx]
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, label

def load_dataset_paths(root_dir, split='train'):
    root_dir = Path(root_dir) / split
    class_names = ['CNV', 'DME', 'DRUSEN', 'NORMAL']
    image_paths, labels = [], []
    for class_idx, class_name in enumerate(class_names):
        class_dir = root_dir / class_name
        if class_dir.exists():
            img_files = list(class_dir.glob('*.jpeg')) + list(class_dir.glob('*.jpg'))
            image_paths.extend([str(f) for f in img_files])
            labels.extend([class_idx] * len(img_files))
    return image_paths, labels, class_names

# --------------------------
# MEMORY OPTIMIZATION FUNCTIONS
# --------------------------
def clear_memory():
    """Clear GPU and CPU memory"""
    torch.cuda.empty_cache()
    gc.collect()

def cleanup_batch(*tensors):
    """Explicitly delete tensors and free memory"""
    for tensor in tensors:
        if tensor is not None:
            del tensor
    torch.cuda.empty_cache()
    gc.collect()

# --------------------------
# GPU SETUP
# --------------------------
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🚀 Using device: {device}")
if torch.cuda.is_available():
    print(f"✅ GPU: {torch.cuda.get_device_name(0)}")

OCT_PATH = './data/OCT2017'

# =================================================================================
# STEP 1: LOAD TEACHER MODEL WITH FEATURE EXTRACTION (FROZEN)
# =================================================================================
print("\n📚 Loading Teacher Model (ResNet-101) with Feature Extraction...")

class ResNet101TeacherWithFeatures(nn.Module):
    def __init__(self, num_classes=4):
        super().__init__()
        self.backbone = models.resnet101(pretrained=False)

        # Get feature dimension before fc layer
        in_features = self.backbone.fc.in_features  # 2048 for ResNet-101

        # Replace fc with custom classifier
        self.backbone.fc = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(in_features, 512),
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(512),
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(256),
            nn.Dropout(0.2),
            nn.Linear(256, num_classes)
        )

        # Feature extraction dimension
        self.feature_dim = in_features  # 2048

    def extract_features(self, x):
        """Extract features before the final classifier"""
        # Forward through backbone layers except fc
        x = self.backbone.conv1(x)
        x = self.backbone.bn1(x)
        x = self.backbone.relu(x)
        x = self.backbone.maxpool(x)

        x = self.backbone.layer1(x)
        x = self.backbone.layer2(x)
        x = self.backbone.layer3(x)
        x = self.backbone.layer4(x)

        x = self.backbone.avgpool(x)
        features = torch.flatten(x, 1)  # [batch_size, 2048]

        return features

    def forward(self, x, return_features=False):
        """
        Forward pass with optional feature return
        Args:
            x: input tensor
            return_features: if True, return (logits, features), else return logits only
        """
        features = self.extract_features(x)
        logits = self.backbone.fc(features)

        if return_features:
            return logits, features
        return logits

# Load pretrained teacher with memory optimization
teacher = ResNet101TeacherWithFeatures(num_classes=4).to(device)
checkpoint = torch.load('/content/best_resnet101_teacher.pth',
                       map_location=device, weights_only=True)
teacher.load_state_dict(checkpoint['model_state_dict'])
teacher.eval()  # Freeze teacher

# Freeze all teacher parameters
for param in teacher.parameters():
    param.requires_grad = False

print(f"✅ Teacher loaded with Val F1: {checkpoint['val_f1']:.4f}")
print(f"   Teacher parameters: {sum(p.numel() for p in teacher.parameters()):,}")
print(f"   Teacher feature dimension: {teacher.feature_dim}")

# Clear checkpoint from memory
del checkpoint
clear_memory()

# =================================================================================
# STEP 2: INITIALIZE STUDENT MODEL WITH FEATURE EXTRACTION (MobileNetV3-Large)
# =================================================================================
print("\n🎓 Initializing Student Model (MobileNetV3-Large) with Feature Extraction...")

class MobileNetV3StudentWithFeatures(nn.Module):
    def __init__(self, num_classes=4, pretrained=True):
        super().__init__()
        # Load pretrained MobileNetV3-Large
        self.backbone = models.mobilenet_v3_large(pretrained=pretrained)

        # Get feature dimension before classifier
        in_features = self.backbone.classifier[0].in_features  # 960 for MobileNetV3-Large

        # Replace classifier
        self.backbone.classifier = nn.Sequential(
            nn.Linear(in_features, 256),
            nn.Hardswish(inplace=True),
            nn.Dropout(0.2),
            nn.Linear(256, num_classes)
        )

        # Feature extraction dimension
        self.feature_dim = in_features  # 960

        # Feature projection layer to match teacher's feature dimension
        # This projects student features (960) to teacher features (2048)
        self.feature_projector = nn.Sequential(
            nn.Linear(self.feature_dim, 1024),
            nn.ReLU(inplace=True),
            nn.Linear(1024, 2048)  # Match teacher feature dim
        )

    def extract_features(self, x):
        """Extract features before the final classifier"""
        x = self.backbone.features(x)
        x = self.backbone.avgpool(x)
        features = torch.flatten(x, 1)  # [batch_size, 960]
        return features

    def forward(self, x, return_features=False):
        """
        Forward pass with optional feature return
        Args:
            x: input tensor
            return_features: if True, return (logits, features, projected_features)
        """
        features = self.extract_features(x)
        logits = self.backbone.classifier(features)

        if return_features:
            projected_features = self.feature_projector(features)
            return logits, features, projected_features
        return logits

student = MobileNetV3StudentWithFeatures(num_classes=4, pretrained=True).to(device)
student_params = sum(p.numel() for p in student.parameters())
print(f"✅ Student initialized")
print(f"   Student parameters: {student_params:,}")
print(f"   Student feature dimension: {student.feature_dim}")
print(f"   Feature projector: {student.feature_dim} → 2048 (teacher dim)")
print(f"   Compression ratio: {sum(p.numel() for p in teacher.parameters()) / student_params:.2f}x smaller")

# =================================================================================
# STEP 3: PREPARE DATASET
# =================================================================================
print("\n📂 Loading OCT Dataset with Proper Split...")

# Load ALL images from Kaggle's training folder
all_paths, all_labels, class_names = load_dataset_paths(OCT_PATH, 'train')

print(f"   Total samples: {len(all_paths):,}")

# Stratified 70/30 split (70% train, 30% temp for val+test)
train_paths, temp_paths, train_labels, temp_labels = train_test_split(
    all_paths, all_labels,
    test_size=0.30,
    stratify=all_labels,
    random_state=42
)

# Split temp into val and test (15% each)
val_paths, test_paths, val_labels, test_labels = train_test_split(
    temp_paths, temp_labels,
    test_size=0.50,
    stratify=temp_labels,
    random_state=42
)

print(f"   Training: {len(train_paths):,} ({len(train_paths)/len(all_paths)*100:.1f}%)")
print(f"   Validation: {len(val_paths):,} ({len(val_paths)/len(all_paths)*100:.1f}%)")
print(f"   Test: {len(test_paths):,} ({len(test_paths)/len(all_paths)*100:.1f}%)")

# Verify class distribution
print(f"\n   Class distribution maintained via stratification")

# Data transforms
train_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomCrop(224),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

val_test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Create datasets and dataloaders
train_dataset = MultiClassOCTDataset(train_paths, train_labels, class_names, train_transform)
val_dataset = MultiClassOCTDataset(val_paths, val_labels, class_names, val_test_transform)
test_dataset = MultiClassOCTDataset(test_paths, test_labels, class_names, val_test_transform)

# Use persistent_workers to prevent memory fragmentation
batch_size = 128 
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,
                         num_workers=2, pin_memory=True, drop_last=True,
                         persistent_workers=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False,
                       num_workers=2, pin_memory=True, persistent_workers=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False,
                        num_workers=2, pin_memory=True, persistent_workers=True)

# =================================================================================
# STEP 4: FEATURE-BASED KNOWLEDGE DISTILLATION LOSS WITH COSINE SIMILARITY
# =================================================================================
class FeatureDistillationLoss(nn.Module):
    """
    Feature-Based Knowledge Distillation Loss with Cosine Similarity

    Combines:
    1. Feature mimicking loss (Cosine similarity between projected student features and teacher features)
    2. Task loss (Cross-entropy with true labels)
    """
    def __init__(self, alpha=0.7, feature_loss_weight=1.0):
        """
        Args:
            alpha: Weight for feature loss vs task loss
            feature_loss_weight: Additional scaling for feature loss
        """
        super().__init__()
        self.alpha = alpha
        self.feature_loss_weight = feature_loss_weight
        self.cosine_loss = nn.CosineEmbeddingLoss()
        self.ce_loss = nn.CrossEntropyLoss()

    def forward(self, student_logits, student_features_projected, teacher_features, labels):
        """
        Args:
            student_logits: Student model predictions [batch_size, num_classes]
            student_features_projected: Projected student features [batch_size, teacher_feature_dim]
            teacher_features: Teacher features [batch_size, teacher_feature_dim]
            labels: True labels [batch_size]
        """
        # Feature mimicking loss using cosine similarity
        # CosineEmbeddingLoss expects target of 1 (similar) or -1 (dissimilar)
        # We want features to be similar, so target = 1 for all pairs
        batch_size = student_features_projected.size(0)
        target = torch.ones(batch_size, device=student_features_projected.device)

        feature_loss = self.cosine_loss(
            student_features_projected,
            teacher_features,
            target
        ) * self.feature_loss_weight

        # Task loss (classification)
        task_loss = self.ce_loss(student_logits, labels)

        # Combined loss
        total_loss = self.alpha * feature_loss + (1 - self.alpha) * task_loss

        return total_loss, feature_loss, task_loss

# Hyperparameters for Feature-Based KD
ALPHA = 0.7  # Weight between feature loss and task loss
FEATURE_LOSS_WEIGHT = 1.0  # Additional scaling for feature loss

kd_criterion = FeatureDistillationLoss(alpha=ALPHA, feature_loss_weight=FEATURE_LOSS_WEIGHT)
print(f"\n  Feature-Based Distillation Configuration:")
print(f"   Alpha (α): {ALPHA} (feature loss weight)")
print(f"   Feature Loss Weight: {FEATURE_LOSS_WEIGHT}")
print(f"   Loss Type: Cosine Similarity Loss")

# =================================================================================
# STEP 5: TRAINING CONFIGURATION
# =================================================================================
optimizer = optim.AdamW(student.parameters(), lr=1e-3, weight_decay=1e-4)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=15, eta_min=1e-6)

num_epochs = 10
best_val_f1 = 0.0
patience = 4
patience_counter = 0

# Use lists of scalars instead of storing large tensors
history = {
    'train_loss': [], 'train_feature': [], 'train_task': [], 'train_acc': [],
    'val_loss': [], 'val_acc': [], 'val_f1': []
}

# =================================================================================
# STEP 6: MEMORY-OPTIMIZED FEATURE DISTILLATION TRAINING LOOP
# =================================================================================
print("\n Starting Feature-Based Knowledge Distillation Training...")
print("=" * 70)

for epoch in range(num_epochs):
    # Clear memory before each epoch
    clear_memory()

    # ----- Training Phase -----
    student.train()
    running_loss, running_feature, running_task = 0.0, 0.0, 0.0
    correct, total = 0, 0

    train_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Train]", leave=False)
    for batch_idx, (inputs, labels) in enumerate(train_bar):
        inputs, labels = inputs.to(device, non_blocking=True), labels.to(device, non_blocking=True)

        optimizer.zero_grad(set_to_none=True)  # More memory efficient

        # Get teacher features and predictions with gradient disabled
        with torch.no_grad():
            _, teacher_features = teacher(inputs, return_features=True)

        # Get student predictions and features
        student_logits, student_features, student_features_projected = student(inputs, return_features=True)

        # Calculate feature-based distillation loss
        loss, feature_loss, task_loss = kd_criterion(
            student_logits, student_features_projected, teacher_features, labels
        )

        loss.backward()
        torch.nn.utils.clip_grad_norm_(student.parameters(), max_norm=1.0)
        optimizer.step()

        # Statistics (convert to float immediately to free tensor memory)
        running_loss += float(loss.item())
        running_feature += float(feature_loss.item())
        running_task += float(task_loss.item())

        with torch.no_grad():
            _, preds = torch.max(student_logits, 1)
            correct += int((preds == labels).sum().item())
            total += int(labels.size(0))

        train_bar.set_postfix({
            'Loss': f"{loss.item():.4f}",
            'Feat': f"{feature_loss.item():.4f}",
            'Acc': f"{100.*correct/total:.2f}%"
        })

        # Memory cleanup after each batch
        cleanup_batch(loss, feature_loss, task_loss, teacher_features,
                     student_logits, student_features, student_features_projected,
                     preds, inputs, labels)

        # Periodically clear memory during training
        if batch_idx % 50 == 0:
            clear_memory()

    train_loss = running_loss / len(train_loader)
    train_feature = running_feature / len(train_loader)
    train_task = running_task / len(train_loader)
    train_acc = 100. * correct / total

    # ----- Validation Phase -----
    student.eval()
    val_loss, val_correct, val_total = 0.0, 0, 0
    all_labels, all_preds = [], []

    with torch.no_grad():
        val_bar = tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Val]", leave=False)
        for inputs, labels in val_bar:
            inputs, labels = inputs.to(device, non_blocking=True), labels.to(device, non_blocking=True)

            _, teacher_features = teacher(inputs, return_features=True)
            student_logits, student_features, student_features_projected = student(inputs, return_features=True)

            loss, _, _ = kd_criterion(student_logits, student_features_projected, teacher_features, labels)

            val_loss += float(loss.item())
            _, preds = torch.max(student_logits, 1)
            val_correct += int((preds == labels).sum().item())
            val_total += int(labels.size(0))

            # Store as numpy arrays immediately to free GPU memory
            all_labels.append(labels.cpu().numpy())
            all_preds.append(preds.cpu().numpy())

            # Clean up batch
            cleanup_batch(loss, teacher_features, student_logits,
                         student_features, student_features_projected,
                         preds, inputs, labels)

    # Concatenate all batches
    all_labels = np.concatenate(all_labels)
    all_preds = np.concatenate(all_preds)

    val_loss /= len(val_loader)
    val_acc = 100. * val_correct / val_total
    _, _, val_f1, _ = precision_recall_fscore_support(
        all_labels, all_preds, average='weighted', zero_division=0
    )

    # Clear validation arrays
    del all_labels, all_preds

    # Update scheduler
    scheduler.step()

    # Save history (only scalars)
    history['train_loss'].append(train_loss)
    history['train_feature'].append(train_feature)
    history['train_task'].append(train_task)
    history['train_acc'].append(train_acc)
    history['val_loss'].append(val_loss)
    history['val_acc'].append(val_acc)
    history['val_f1'].append(val_f1)

    # Print epoch summary
    print(f"\n Epoch {epoch+1} Summary:")
    print(f"   Train - Total: {train_loss:.4f} | Feature: {train_feature:.4f} | Task: {train_task:.4f} | Acc: {train_acc:.2f}%")
    print(f"   Val - Loss: {val_loss:.4f} | Acc: {val_acc:.2f}% | F1: {val_f1:.4f}")

    # Save best model
    if val_f1 > best_val_f1:
        best_val_f1 = val_f1
        patience_counter = 0

        # Save with memory optimization
        checkpoint = {
            'epoch': epoch + 1,
            'model_state_dict': student.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'val_f1': val_f1,
            'val_acc': val_acc,
            'alpha': ALPHA,
            'feature_loss_weight': FEATURE_LOSS_WEIGHT,
            'class_names': class_names
        }
        torch.save(checkpoint, 'best_mobilenetv3_student_feature_kd.pth')
        del checkpoint
        print(f"   💾 Model saved with F1: {val_f1:.4f}")
    else:
        patience_counter += 1
        print(f"   ⏳ No improvement for {patience_counter}/{patience} epochs")

    if patience_counter >= patience:
        print(f"\n⏹️  Early stopping at epoch {epoch+1}")
        break

    # Clear memory after each epoch
    clear_memory()

print("\n" + "="*70)
print(f"✅ Feature-Based Knowledge Distillation Complete!")
print(f"🏆 Best Validation F1: {best_val_f1:.4f}")

# =================================================================================
# STEP 7: FINAL TEST EVALUATION
# =================================================================================
print("\n🧪 Evaluating Student on Test Set...")

# Load best student model
checkpoint = torch.load('best_mobilenetv3_student_feature_kd.pth', map_location=device, weights_only=True)
student.load_state_dict(checkpoint['model_state_dict'])
student.eval()

# Evaluate student
test_correct, test_total = 0, 0
all_test_labels, all_test_preds = [], []

with torch.no_grad():
    test_bar = tqdm(test_loader, desc="Testing Student")
    for inputs, labels in test_bar:
        inputs, labels = inputs.to(device, non_blocking=True), labels.to(device, non_blocking=True)
        outputs = student(inputs)
        _, preds = torch.max(outputs, 1)
        test_correct += int((preds == labels).sum().item())
        test_total += int(labels.size(0))

        # Store as numpy to save memory
        all_test_labels.append(labels.cpu().numpy())
        all_test_preds.append(preds.cpu().numpy())

        # Clean up
        cleanup_batch(outputs, preds, inputs, labels)

# Concatenate results
all_test_labels = np.concatenate(all_test_labels)
all_test_preds = np.concatenate(all_test_preds)

test_acc = 100. * test_correct / test_total
test_precision, test_recall, test_f1, _ = precision_recall_fscore_support(
    all_test_labels, all_test_preds, average='weighted', zero_division=0
)

print(f"\n📊 Student Test Results:")
print(f"   Accuracy:  {test_acc:.2f}%")
print(f"   Precision: {test_precision:.4f}")
print(f"   Recall:    {test_recall:.4f}")
print(f"   F1-Score:  {test_f1:.4f}")

print("\n📋 Classification Report:")
print(classification_report(all_test_labels, all_test_preds,
                           target_names=class_names, digits=4))

# Plotting functions (optional - can be commented out if memory is tight)
try:
    import matplotlib.pyplot as plt
    import seaborn as sns

    # Confusion Matrix
    cm = confusion_matrix(all_test_labels, all_test_preds)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=class_names, yticklabels=class_names)
    plt.title('Student Model (MobileNetV3 + Feature KD) - Confusion Matrix')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.tight_layout()
    plt.savefig('student_feature_kd_confusion_matrix.png', dpi=300)
    plt.close()  # Close figure to free memory
    print("📊 Confusion matrix saved and figure closed")

except ImportError:
    print("⚠️  Matplotlib/Seaborn not available for plotting")

# Clear test arrays
del all_test_labels, all_test_preds, checkpoint
clear_memory()

# =================================================================================
# STEP 8: CALCULATE FISHER INFORMATION MATRIX
# =================================================================================
print("\n🧮 Computing Fisher Information Matrix for EWC...")

def compute_fisher_information(model, dataloader, num_samples=500):  # Reduced samples
    """
    Memory-optimized Fisher Information computation
    """
    model.eval()
    fisher = {}

    # Initialize fisher with zeros on CPU
    for n, p in model.named_parameters():
        if p.requires_grad:
            fisher[n] = torch.zeros_like(p, device='cpu')

    samples_processed = 0
    criterion = nn.CrossEntropyLoss()

    pbar = tqdm(dataloader, desc="Computing Fisher",
                total=min(num_samples//dataloader.batch_size, len(dataloader)))

    for inputs, labels in pbar:
        if samples_processed >= num_samples:
            break

        inputs, labels = inputs.to(device, non_blocking=True), labels.to(device, non_blocking=True)

        model.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()

        # Accumulate squared gradients on CPU
        for n, p in model.named_parameters():
            if p.requires_grad and p.grad is not None:
                fisher[n] += p.grad.detach().cpu().pow(2) * inputs.size(0)

        samples_processed += inputs.size(0)

        # Clean up
        cleanup_batch(outputs, loss, inputs, labels)

        # Clear memory periodically
        if samples_processed % 200 == 0:
            clear_memory()

    # Normalize by number of samples
    for n in fisher:
        fisher[n] /= samples_processed

    return fisher

# Compute Fisher on Task A (OCT) training data
fisher_dict = compute_fisher_information(student, train_loader, num_samples=500)

# Save optimal parameters θ*_A and Fisher matrix F_A for Phase 3
print("\n💾 Saving Task A parameters and Fisher Information for EWC...")
checkpoint = {
    'model_state_dict': student.state_dict(),
    'fisher_information': fisher_dict,
    'task': 'OCT',
    'test_f1': test_f1,
    'test_acc': test_acc,
    'alpha': ALPHA,
    'feature_loss_weight': FEATURE_LOSS_WEIGHT,
    'class_names': class_names,
    'distillation_type': 'feature_based_cosine'
}
torch.save(checkpoint, 'task_a_student_optimal_feature_kd.pth')
del checkpoint
clear_memory()

print("✅ Fisher Information Matrix saved")
print(f"   Parameters stored: {len(fisher_dict)}")

# =================================================================================
# STEP 9: COMPARE TEACHER VS STUDENT
# =================================================================================
print("\n" + "="*70)
print("📊 PHASE 2 FINAL COMPARISON (FEATURE-BASED KD WITH COSINE LOSS)")
print("="*70)

# Load teacher checkpoint just for metrics
teacher_checkpoint = torch.load('/content/best_resnet101_teacher.pth',
                               map_location='cpu', weights_only=True)

print(f"\n🎓 Teacher (ResNet-101):")
print(f"   Parameters: {sum(p.numel() for p in teacher.parameters()):,}")
print(f"   Test F1: {teacher_checkpoint['val_f1']:.4f}")
print(f"   Feature Dimension: 2048")

print(f"\n🎓 Student (MobileNetV3 + Feature KD):")
print(f"   Parameters: {student_params:,}")
print(f"   Test F1: {test_f1:.4f}")
print(f"   Test Acc: {test_acc:.2f}%")
print(f"   Feature Dimension: 960 → 2048 (projected)")

compression = sum(p.numel() for p in teacher.parameters()) / student_params
f1_retention = (test_f1 / teacher_checkpoint['val_f1']) * 100

print(f"\n📈 Feature-Based Knowledge Distillation Efficiency:")
print(f"   Model size: {compression:.1f}x smaller")
print(f"   F1 retention: {f1_retention:.2f}%")
print(f"   Distillation method: Feature mimicking (Cosine Similarity)")

# Clean up
del teacher_checkpoint
clear_memory()

print("\n🎯 Phase 2 Complete! Ready for Phase 3 (EWC Continual Learning)")
print("="*70)

🚀 Using device: cuda
✅ GPU: Tesla T4

📚 Loading Teacher Model (ResNet-101) with Feature Extraction...
✅ Teacher loaded with Val F1: 0.9693
   Teacher parameters: 43,683,140
   Teacher feature dimension: 2048

🎓 Initializing Student Model (MobileNetV3-Large) with Feature Extraction...
Downloading: "https://download.pytorch.org/models/mobilenet_v3_large-8738ca79.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v3_large-8738ca79.pth


100%|██████████| 21.1M/21.1M [00:00<00:00, 139MB/s]


✅ Student initialized
   Student parameters: 6,302,260
   Student feature dimension: 960
   Feature projector: 960 → 2048 (teacher dim)
   Compression ratio: 6.93x smaller

📂 Loading OCT Dataset with Proper Split...
   Total samples: 83,484
   Training: 58,438 (70.0%)
   Validation: 12,523 (15.0%)
   Test: 12,523 (15.0%)

   Class distribution maintained via stratification

⚙️  Feature-Based Distillation Configuration:
   Alpha (α): 0.7 (feature loss weight)
   Feature Loss Weight: 1.0
   Loss Type: Cosine Similarity Loss

🔬 Starting Feature-Based Knowledge Distillation Training...


Epoch 1/10 [Train]:   0%|          | 0/456 [00:00<?, ?it/s]

Epoch 1/10 [Val]:   0%|          | 0/98 [00:00<?, ?it/s]

...


📊 Epoch 10 Summary:
   Train - Total: 0.0207 | Feature: 0.0136 | Task: 0.0371 | Acc: 98.66%
   Val - Loss: 0.0840 | Acc: 94.92% | F1: 0.9486
   ⏳ No improvement for 2/4 epochs

✅ Feature-Based Knowledge Distillation Complete!
🏆 Best Validation F1: 0.9675

🧪 Evaluating Student on Test Set...


Testing Student:   0%|          | 0/98 [00:00<?, ?it/s]


📊 Student Test Results:
   Accuracy:  96.83%
   Precision: 0.9690
   Recall:    0.9683
   F1-Score:  0.9685

📋 Classification Report:
              precision    recall  f1-score   support

         CNV     0.9823    0.9744    0.9783      5581
         DME     0.9791    0.9377    0.9580      1702
      DRUSEN     0.8774    0.9358    0.9057      1293
      NORMAL     0.9759    0.9835    0.9797      3947

    accuracy                         0.9683     12523
   macro avg     0.9537    0.9579    0.9554     12523
weighted avg     0.9690    0.9683    0.9685     12523

📊 Confusion matrix saved and figure closed

🧮 Computing Fisher Information Matrix for EWC...


Computing Fisher:   0%|          | 0/3 [00:00<?, ?it/s]


💾 Saving Task A parameters and Fisher Information for EWC...
✅ Fisher Information Matrix saved
   Parameters stored: 178

📊 PHASE 2 FINAL COMPARISON (FEATURE-BASED KD WITH COSINE LOSS)

🎓 Teacher (ResNet-101):
   Parameters: 43,683,140
   Test F1: 0.9693
   Feature Dimension: 2048

🎓 Student (MobileNetV3 + Feature KD):
   Parameters: 6,302,260
   Test F1: 0.9685
   Test Acc: 96.83%
   Feature Dimension: 960 → 2048 (projected)

📈 Feature-Based Knowledge Distillation Efficiency:
   Model size: 6.9x smaller
   F1 retention: 99.92%
   Distillation method: Feature mimicking (Cosine Similarity)

🎯 Phase 2 Complete! Ready for Phase 3 (EWC Continual Learning)
